# MOOS_FSM


## Setup

In [1]:
from enum import Enum

## Class Creation

In [2]:
class Mode(Enum):
    
    LEOP = 0
    IDLE = 1
    IMAGING = 2
    PROCESSING = 3
    DOWNLINK = 4

In [3]:
class SpacecraftState:
    def __init__(self):

        # Change State Variables

        self.curr_imaging = False
        self.curr_downlinking = False
        self.curr_processing = False

        self.battery_level = 0.8
        self.location = True
        self.sunlight = 0.5
        self.reaction_wheel_saturation = 0.0
        self.angular_rate = 0.0
        self.next_imaging = 20000.0
        self.next_comm = 20000.0
        self.data_processed = True
        self.curr_imaging = False
        self.curr_downlinking = False
        self.curr_processing = False


        #Inside State Variables

        # LEOP

        self.Deployed = True
        self.Power = True
        self.detumbled = True
        self.UHFRx_Antenna = True
        self.Wait_Period = True

        # Imaging

        self.pre_imaging = True
        self.Calibration = True
        self.ADCS_slew = True
        self.side_slither = True
        self.imaging = True
        
        # Preprocessing

        self.storage = True
        self.compression = True
        
        # Downlink

        self.DSP = True
        self.Signal_Chain = True
        self.TX = True



        

In [4]:
class FINCH:
    
    def __init__(self):
        self.state = SpacecraftState()
        self.mode = Mode.LEOP

        self.print_mode()
        
    def refresh_mode(self):
        
        if self.change_mode():
            pass
        else:
            self.mode_work()
            
    def mode_work(self):
        
        if self.mode == Mode.IDLE:
                
            if (self.state.reaction_wheel_saturation != 0.0 and self.state.battery_level > 60) or (self.state.reaction_wheel_saturation < 0.5 and self.state.sunlight > 0.0):
                print("Desaturating reaction wheels")
                self.print_mode()
            else:
                print("Pointing in the direction of the sun")
                self.print_mode()
                    
        elif self.mode == Mode.PROCESSING:
                print("Processing")
                self.print_mode()
            
        elif self.mode == Mode.IMAGING:
                print("Imaging")
                self.print_mode()
            
        elif self.mode == Mode.DOWNLINK:
                print("Downlinking")
                self.print_mode()
                
    def change_mode(self):
        
        if self.mode == Mode.LEOP: 
            if self.state.sunlight and self.state.angular_rate == 0.0:
                self.mode = Mode.IDLE
                self.print_mode()
                return True
            
        elif self.mode == Mode.IDLE:
            if self.IDLE_IMAGING():
                self.mode = Mode.IMAGING
                self.print_mode()
                return True
            elif self.IDLE_DOWNLINK():
                self.mode = Mode.DOWNLINK
                self.print_mode()
                return True
            elif self.state.data_processed == False and self.state.next_comm > 1000 and self.state.next_imaging > 1000 and self.state.battery_level > 0.6:
                self.mode = Mode.PROCESSING
                self.print_mode()
                return True

        elif self.mode == Mode.IMAGING:
            
            if self.IMAGING_PROCESSING():
                self.mode = Mode.PROCESSING
                self.print_mode()
                return True

            if self.IMAGING_IDLE():
                self.mode = Mode.IDLE
                self.print_mode()
                return True

        elif self.mode == Mode.DOWNLINK:
            
            if self.DOWNLINK_IDLE():
                self.mode = Mode.IDLE
                self.print_mode()
                return True

        elif self.mode == Mode.PROCESSING:
            
            if self.PRCOESSING_IDLE():
                self.mode = Mode.IDLE
                self.print_mode()
                return True
            
        return False


    def PROCESSING_IDLE(self):

        total = 1
        sum = 0

        if not self.state.curr_processing:

            sum += 1

        if sum == total:
            return True
        else:
            return False

    def IMAGE_IDLE(self):

        total = 1
        sum = 0

        if not self.state.curr_imaging:

            sum += 1

        if sum == total:
            return True
        else:
            return False
    
    def DOWNLINK_IDLE(self):

        total = 1
        sum = 0

        if not self.state.curr_downlinking:

            sum += 1

        if sum == total:
            return True
        else:
            return False

    def IMAGE_PROCESSING(self):

        total = 2
        sum = 0

        if not self.state.curr_imaging:

            sum += 1

        if self.state.battery_level > 0.6:

            sum += 1

        self.state.processing = True

        if sum == total:
            return True
        else:
            return False

    def IDLE_IMAGING(self):

        total = 4
        sum = 0

        if self.state.angular_rate == 0.0:

            sum += 1

        if self.state.next_imaging < 1000: 

            sum += 1

        if self.state.battery_level > 0.75:

            sum += 1

        if self.state.reaction_wheel_saturation == 0.0:

            sum += 1

        self.state.curr_imaging = True

        if sum == total:
            return True
        else:
            return False

    def IDLE_DOWNLINK(self):

        total = 3
        sum = 0

        if self.state.data_processed == True: 

            sum += 1
            
        
        if self.state.next_comm < 1000: 
            
            sum += 1
            
            
        if self.state.battery_level > 0.7:

            sum += 1

        self.state.curr_downlinking = True

        if sum == total:
            return True
        else:
            return False

    def IDLE_PROCESSING(self):
        
        total = 4
        sum = 0

        if self.state.data_processed == False: 
            
            sum += 1

        if self.state.next_comm > 1000: 
            
            sum += 1
            
        if self.state.next_imaging > 1000:
        
            sum += 1
        
        if self.state.battery_level > 0.6:

            sum += 1

        if sum == total:
            return True
        else:
            return False
    
    def print_mode(self):
        
        print("Current Mode is " + str(self.mode))
                
        

In [5]:
Mingde = FINCH()
Mingde.refresh_mode()
Mingde.state.battery_level = 0.4
Mingde.refresh_mode()
Mingde.state.battery_level = 0.9
Mingde.refresh_mode()

Current Mode is Mode.LEOP
Current Mode is Mode.IDLE
Desaturating reaction wheels
Current Mode is Mode.IDLE
Desaturating reaction wheels
Current Mode is Mode.IDLE
